In [5]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=440aeb6d7820a959359eac8b845f717782a183551c61466909b3f5d585162c69
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [1]:
import os
import json
import pyspark
# import logging
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

# import logging




spark = SparkSession.builder.getOrCreate()

In [2]:
#Necessay Function
def get_batter_bowler_striker(column, column_need):
  if column_need.strip() == 'batter':
    return column['batter']
    print(column)
  elif column_need.strip() == 'bowler':
    return column['bowler']
    print(column)
  elif column_need.strip() == 'non_striker':
    return column['non_striker']
    print(column)

#Necessay Function
def get_run_ball_by_ball(column):

  # column = column['runs']
  # # column = dict(column)
  runs_list = [-1]*3
  # logger.info("this is the column", column)

  for scores in column.keys():
    if scores == 'runs':
      return column[scores]


  return runs_list

def get_info_and_meta_data(all_information):
  for data in all_information.keys():
    if data == 'meta':
      meta_information = all_information['meta']
    elif data == 'info':
      info_information = all_information["info"]
  return meta_information, info_information

def get_necessary_information(data):
  date = data['dates'][0]

  if "city" in data.keys():
    city = data['city']
  else:
    city = "NULL"
  event_name = data['event']['name']

  if 'match_number' in data['event'].keys():
    match_number = data['event']['match_number']
  else:
    match_number = "NULL"
  gender = data['gender']

  if "winner" in data['outcome'].keys():

    winner_team = data['outcome']['winner']
  elif 'result' in data['outcome'].keys():

    winner_team = data['outcome']['result']

  if "winner" in data['outcome'].keys():
    if 'wickets' in data['outcome']['by'].keys():
      winned_by = f"{data['outcome']['by']['wickets']} wickets"
    elif 'runs' in data['outcome']['by'].keys():
      winned_by = f"{data['outcome']['by']['runs']} runs"
  else:
    winned_by = "NULL"



  team_1 = data['teams'][0]
  team_2 = data['teams'][1]
  return date, city, event_name, match_number, gender, winner_team, winned_by, team_1, team_2


# print(get_run_ball_by_ball({'batter': 'L Lee', 'bowler': 'Panna Ghosh', 'non_striker': 'A Steyn', 'runs': {'batter': 0, 'extras': 0, 'total': 0}}))

get_run_ball_by_ball_udf =  udf(lambda column: get_run_ball_by_ball(column), StringType())
get_batter_bowler_striker_udf =  udf(lambda column, column_need: get_batter_bowler_striker(column, column_need), StringType())

In [16]:

all_the_files = os.listdir("/female_dataset")
print(all_the_files)
necessary_columns = []
for i in all_the_files:
  # print(f"/female_dataset/{i}")
  if i.endswith(".json"):
    with open(f"/female_dataset/{i}","r") as file_obj:
      file_content = file_obj.read()
      # print(file_content)
      details = json.loads(file_content)
      meta_information, info_information = get_info_and_meta_data(details)
      date, city, event_name, match_number, gender, winner_team, winned_by, team_1, team_2 = get_necessary_information(info_information)
      necessary_columns = []
      temp_dict = {}
      for i in range(len(details['innings'][0]['overs'])):
        temp_dict['overs'] = i
        temp_dict['balls_per_over'] = details['innings'][0]['overs'][i]['deliveries']
        # print(student_details['innings'][0]['overs'][i]['deliveries'])
        necessary_columns.append(temp_dict)
        temp_dict = {}
      dataframe = spark.createDataFrame(necessary_columns)
      # dataframe.show(1000, False)
      dataframe = dataframe.select("overs", posexplode_outer(dataframe.balls_per_over))

      dataframe_new = dataframe.withColumn("BATTER", get_batter_bowler_striker_udf(col("col"), lit("batter")))
      dataframe_new = dataframe_new.withColumn("BOWLER", get_batter_bowler_striker_udf(col("col"), lit("bowler")))
      dataframe_new = dataframe_new.withColumn("NON_STRIKER", get_batter_bowler_striker_udf(col("col"), lit("non_striker")))
      dataframe_new = dataframe_new.withColumn("runs_scored_per_ball", get_run_ball_by_ball_udf(col("col")))
      dataframe_new = dataframe_new.select('*', lit(date).alias("MATCH_DATE"), lit(city).alias("MATCH_CITY"),\
                                           lit(event_name).alias("EVENT_NAME"),lit(match_number).alias("MATCH_NUMBER"),lit(gender).alias("GENDER"),\
                                           lit(winner_team).alias("WINNER_TEAM"),lit(winned_by).alias("WINNED_BY"),lit(team_1).alias("FIRST_TEAM"),lit(team_2).alias("SECOND_TEAM"))

      dataframe_new = dataframe_new.withColumn("runs_scored_per_ball", regexp_replace(col("runs_scored_per_ball"), "(\{extras=)|(total=)|(batter=)|(\})", "")).withColumn("EXTRAS_EARNED_PER_BALL", trim(split(col("runs_scored_per_ball"), ',').getItem(0))).withColumn("TOTAL_RUNS_PER_BALL", trim(split(col("runs_scored_per_ball"), ',').getItem(1))).withColumn("BATTER_SCORED_RUNS_PER_BALL", trim(split(col("runs_scored_per_ball"), ',').getItem(2)))
      dataframe_new = dataframe_new.select('overs','FIRST_TEAM', 'SECOND_TEAM','EVENT_NAME','MATCH_DATE','MATCH_CITY','MATCH_NUMBER','GENDER','WINNER_TEAM','WINNED_BY','BATTER','BOWLER','NON_STRIKER','BATTER_SCORED_RUNS_PER_BALL','TOTAL_RUNS_PER_BALL', 'EXTRAS_EARNED_PER_BALL')
      # dataframe_new.write.parquet("/output_folder/")\
      # dataframe_new.write.mode("append").format("parquet").save("/output_folder/")

      dataframe_new.show(10, False)
      break


['1301329.json', '1041523.json', '1260105.json', '1161229.json', '603239.json', '1126728.json', '1260095.json', '1277095.json', '757509.json', '1202249.json', '1289033.json', '1131778.json', '1273422.json', '923327.json', '1243932.json', '808865.json', '1043961.json', '1375874.json', '1345092.json', '627007.json', '271465.json', '276226.json', '271466.json', '1344504.json', '1263617.json', '1244845.json', '1310985.json', '1156212.json', '1085960.json', '1243931.json', '1243912.json', '1328486.json', '1244843.json', '828245.json', '1085954.json', '644943.json', '946545.json', '1249233.json', '1301337.json', '1319713.json', '1131233.json', '1126718.json', '1085963.json', '1126727.json', '1263577.json', '584546.json', '757505.json', '1208344.json', '1344505.json', '527679.json', '1301330.json', '594914.json', '668513.json', '935823.json', '962017.json', '1260096.json', '1085946.json', '1085962.json', '542852.json', '1073429.json', '1392350.json', '1085970.json', '1260103.json', '1249234.j